<a href="https://colab.research.google.com/github/NC25/gym_fishing/blob/master/abstract_models/model_based_dynamic_programming.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Dynamic Programming breaks down an objective in subparts and solves those subparts to find a total solution

Assume that our model is perfect

Features:


*   Updates policty based on Bellman's Equation
*   Objective: Improve the policy to make it greedy
*   Policy iteration - iterates evalution and improvement to get optimal policu
*   Value iteration
*   Generalized policy iteration 












# Policy Evaluation

In [ ]:
!git clone https://github.com/dennybritz/reinforcement-learning.git

In [ ]:
'''
from google.colab import drive
drive.mount('/content/gdrive')
'''

In [ ]:
%load gridworld.py

In [ ]:
import numpy as np
import pprint
import sys
if "../" not in sys.path:
  sys.path.append("../") 

import tensorflow as tf

path = '/content/gridworld.py'

from gridworld import GridworldEnv
import plotting

In [ ]:
pp = pprint.PrettyPrinter(indent=2)
env = GridworldEnv()

In [ ]:
def policy_eval(policy, env, discount_factor=1.0, theta=0.00001):


    # Set value function as a set of 0s
    Value = np.zeros(env.nS)
    while True:
        delta = 0 #delta is the gradient propogator
        #make mini v values
        for s in range(env.nS):
            v = 0
            # Look at the possible next actions
            for a, action_prob in enumerate(policy[s]):
                # For each action, look at the next possible states
                for  prob, next_state, reward, done in env.P[s][a]:
                    # Calculate expected value
                    v += action_prob * prob * (reward + discount_factor * Value[next_state])

            #v = (s, a , s_t+1)

            # Determine variation in value function
            delta = max(delta, np.abs(v - Value[s]))
            Value[s] = v
        # Stop evaluating once our value function change is below a threshold (theta)
        if delta < theta:
            break
    return np.array(Value)

    





In [ ]:
random_policy = np.ones([env.nS, env.nA]) / env.nA
v = policy_eval(random_policy, env)

In [ ]:
print("Value Function".format(v))
print(v)

Value Function
[  0.         -13.99993529 -19.99990698 -21.99989761 -13.99993529
 -17.9999206  -19.99991379 -19.99991477 -19.99990698 -19.99991379
 -17.99992725 -13.99994569 -21.99989761 -19.99991477 -13.99994569
   0.        ]


In [ ]:
expected_v = np.array([0, -14, -20, -22, -14, -18, -20, -20, -20, -20, -18, -14, -22, -20, -14, 0])
np.testing.assert_array_almost_equal(v, expected_v, decimal=2)

# Policy Iteration

In [ ]:
def policy_improvement(env, policy_eval_fn=policy_eval, discount_factor=1.0):

  def action_value(state, Value):
    #determines value of an action at a different state
    current_action = np.zeros(env.nA)
    for a in range(env.nA):
      for prob, next_state, reward, done in env.P[state][a]:
        current_action[a] += prob * (reward + discount_factor * Value[next_state])
    return current_action

  #start with a random policy
  policy = np.ones([env.nS, env.nA]) / env.nA

  while True:
     Value = policy_eval_fn(random_policy, env)

     #Will be set to false when we make a change
     policy_stable = True

     #For each state, choose the best action

     for s in range(env.nS):
       chosen_action = np.argmax(policy[s]) #best action is equal the action of the optimal policy

       #find best action
       sample_action = action_value(s, Value)
       best_action = np.argmax(action_value)
       #np.argmax returns the index of the max value

       #Implement Greedy policy
       if chosen_action != best_action:
         policy_stable = False
       policy[s] = np.eye(env.nA)[best_action]
     
  if policy_stable == True:
      return chosen_action, Value    







In [ ]:
policy, v = policy_improvement(env)

KeyboardInterrupt: ignored

# Value Iteration

In [ ]:
def value_iteration(env, theta=0.0001,  discount_factor=1.0):

  def best_action(state, Value):
    action = np.zeroes(env.nA)
    for a in range(env.nA):
      for prob, next_state, reward, done in env.P[state][a]:
        current_action[a] += prob * (reward + discount_factor * Value[s])
    return current_action


    Value = np.zeros(env.nS)

    while True:

      delta = 0
      for s in range(env.nS):
        #find the next best action
        sample_action = best_action(state, Value)
        best_action = np.max(current_action)
        #calculate delta
        delta = max(delta, np.abs(best_action - Value[s]))
        #update value function
        V[s] = best_action
      if delta < theta:
        break



      #implement determinstic policy
      policy = np.zeros([env.nS, env.nA])
      for s in range(env.nS):
        #find best action for state
        current_action = best_action(state, Value)
        best_action = np.argmax(current_action)
        policy[s, best_action] = 1.0

      return policy, Value






In [ ]:
policy, Value = value_iteration(env)